In [13]:
import torch
from transformers import AutoTokenizer, AutoModel
from peft import PeftModel

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("saved_model")

# Load base model
base_model_name = "sentence-transformers/all-MiniLM-L6-v2"
base_model = AutoModel.from_pretrained(base_model_name)

# Load PEFT model
model = PeftModel.from_pretrained(base_model, "saved_model")
model.eval()

def calculate_match_score(resume_text, job_role):
    # Tokenize inputs separately
    resume_inputs = tokenizer(resume_text, padding=True, truncation=True, return_tensors="pt")
    job_inputs = tokenizer(job_role, padding=True, truncation=True, return_tensors="pt")

    with torch.no_grad():
        # Use the base model to get embeddings
        resume_outputs = model.base_model(
            input_ids=resume_inputs['input_ids'], 
            attention_mask=resume_inputs['attention_mask']
        )
        job_outputs = model.base_model(
            input_ids=job_inputs['input_ids'], 
            attention_mask=job_inputs['attention_mask']
        )

        # Use the pooler output for similarity calculation
        resume_embeddings = resume_outputs.pooler_output
        job_embeddings = job_outputs.pooler_output

    # Compute cosine similarity
    similarity_score = torch.nn.functional.cosine_similarity(resume_embeddings, job_embeddings)

    return float(similarity_score.mean()) * 100  # Convert to percentage

In [10]:
print(model)


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 384, padding_idx=0)
        (position_embeddings): Embedding(512, 384)
        (token_type_embeddings): Embedding(2, 384)
        (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-5): 6 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): lora.Linear(
                  (base_layer): Linear(in_features=384, out_features=384, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=384, out_features=8, bias=False)
                  )
          

In [11]:
import torch

def calculate_match_score(resume_text, job_role):
    # Tokenize inputs separately
    resume_inputs = tokenizer(resume_text, padding=True, truncation=True, return_tensors="pt")
    job_inputs = tokenizer(job_role, padding=True, truncation=True, return_tensors="pt")

    with torch.no_grad():
        # Get embeddings from pooler_output instead of logits
        resume_outputs = model(**resume_inputs).pooler_output
        job_outputs = model(**job_inputs).pooler_output

    # Compute cosine similarity
    similarity_score = torch.nn.functional.cosine_similarity(resume_outputs, job_outputs)

    return float(similarity_score.mean()) * 100  # Convert to percentage


In [14]:
resume_sample = "Software engineer with 5 years of experience in Python and ML."
job_role_sample = "Looking for a Python machine learning engineer."

score = calculate_match_score(resume_sample, job_role_sample)
print(f"Match Score: {score:.2f}%")

Match Score: 86.71%
